<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [12]:
from torch_geometric.datasets import TUDataset, GNNBenchmarkDataset
from torch_geometric import utils
import networkx as nx
import numpy as np
from tqdm.notebook import tqdm

In [13]:
all_datasets = ["MUTAG","ENZYMES","PROTEINS","CIFAR10","MNIST","COLLAB","IMDB-BINARY","REDDIT-BINARY"]

In [15]:
statistics = dict()
for dataset_txt in all_datasets[:2]:
    if dataset_txt in GNNBenchmarkDataset.names:
        dataset = GNNBenchmarkDataset(root='data_colab/GNNBenchmarkDataset', name=dataset_txt)
    else:
        dataset = TUDataset(root='data_colab/TUDataset',name=dataset_txt)
        
    nxdata = {
    'Assortativity':np.empty(len(dataset)),
    'Triangles':np.empty(len(dataset)),
    'Transitivity':np.empty(len(dataset)),
    'Clustering':np.empty(len(dataset))
    }
    for i, graph in enumerate(tqdm(dataset, desc=dataset_txt)):
        G = utils.to_networkx(graph, to_undirected=True)
        nxdata['Assortativity'][i] = nx.degree_assortativity_coefficient(G)
        nxdata['Triangles'][i] = np.sum(list(nx.triangles(G).values()))/3
        nxdata['Transitivity'][i] = nx.transitivity(G)
        nxdata['Clustering'][i] = nx.average_clustering(G)
        
    statistics[dataset_txt] = nxdata.copy()


MUTAG:   0%|          | 0/188 [00:00<?, ?it/s]

ENZYMES:   0%|          | 0/600 [00:00<?, ?it/s]

In [51]:
for dt in statistics:
    print(dt,'-----------------')
    for metric in statistics[dt]:
        nans = np.isnan(statistics[dt][metric])
        mean = statistics[dt][metric][~nans].mean()
        print(f"\t{metric:>13}: {mean:>9.4f} +- {statistics[dt][metric].std():<9.4f} || nans: {np.sum(nans)}")
    print()

MUTAG -----------------
	Assortativity:   -0.2787 +- 0.1689    || nans: 0
	    Triangles:    0.0000 +- 0.0000    || nans: 0
	 Transitivity:    0.0000 +- 0.0000    || nans: 0
	   Clustering:    0.0000 +- 0.0000    || nans: 0

ENZYMES -----------------
	Assortativity:   -0.0080 +- nan       || nans: 3
	    Triangles:   25.5100 +- 12.1648   || nans: 0
	 Transitivity:    0.4209 +- 0.1691    || nans: 0
	   Clustering:    0.4534 +- 0.1978    || nans: 0

